In [1]:
from numpy import linalg as la
from numpy import *
# 用户-菜肴矩阵，有很多菜都没有评分
# 行是用户，列是菜肴，其实正常情况下是用户数量远大于菜的数量
U,Sigma,VT=la.svd(mat([[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]))
Sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

In [2]:
Sig2=Sigma**2
# 计算总能量
print(sum(Sig2))
# 计算总能量的90%
print(sum(Sig2)*0.9)
# 计算前两个元素所包含的能量
print(sum(Sig2[:2]))
# 前两个元素能量不够，所以计算前三个元素所包含的能量
print(sum(Sig2[:3]))

541.9999999999995
487.7999999999996
378.8295595113579
500.50028912757926


In [4]:
# 推荐引擎一般是根据相似度计算，相似度高的就认为喜欢
# 一般是欧式距离，皮尔逊相关系数，余弦相似度三种
from numpy import *
from numpy import linalg as la
# inA和inB都是列向量
def ecludSim(inA, inB) :
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA, inB) :
    # 检查是否存在三个或更多的点，若不存在，则返回1.0，这是因为此时两个向量完全相关
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA, inB) :
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

# 一般物品比用户少，所以我们计算物品之间的相似度

In [5]:
myMat=mat([[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]])

In [6]:
# 构建SVD推荐引擎，在第三行对数据集记性了SVD分解
def svdEst(dataMat, user, simMeas, item) :
    n = shape(dataMat)[1]
    # 得到数据集中物品的数目
    
    # 对评估值进行初始化
    simTotal = 0.0; ratSimTotal = 0.0
    
    U,Sigma,VT = la.svd(dataMat)
    
    # 使用奇异值构建一个对角矩阵
    Sig4 = mat(eye(4)*Sigma[:4])
    
    # 利用U矩阵将物品转换到低维空间中
    xformedItems = dataMat.T * U[:, :4] * Sig4.I
    
    # 对于给定的用户，for循环在用户对应行的所有元素上进行遍历，
    # 这里的相似度是在低维空间下进行的。相似度的计算方法也会作为一个参数传递给该函数
    for j in range(n) :
        userRating = dataMat[user,j]
        if userRating == 0 or j == item : continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        # print便于了解相似度计算的进展情况
        print('the %d and %d similarity is : %f' % (item, j, similarity))
        
        # 对相似度求和
        simTotal += similarity
        
        # 对相似度及评分值的乘积求和
        ratSimTotal += similarity * userRating
    if simTotal == 0 : return 0
    else : return ratSimTotal/simTotal
    # 通过除以所有的评分总和，对上述相似度评分的乘积进行归一化。这使得评分值在0-5之间，
    # 而这些评分值则用于对预测值进行排序

In [7]:
# 用来计算在给定相似度计算方法的条件下，用户对物品的估计评分值
# 参数：数据矩阵、用户编号、物品编号、相似度计算方法，矩阵采用图1和图2的形式
# 即行对应用户、列对应物品
def standEst(dataMat, user, simMeas, item) :
    # 首先得到数据集中的物品数目
    n = shape(dataMat)[1]
    # 对两个用于计算估计评分值的变量进行初始化
    simTotal = 0.0; ratSimTotal = 0.0
    # 遍历行中的每个物品
    for j in range(n) :
        userRating = dataMat[user,j]
        # 如果某个物品评分值为0，意味着用户没有对该物品评分，跳过
        if userRating == 0 : continue
        # 寻找两个用户都评级的物品，变量overLap给出的是两个物品当中已经被评分的那个元素
        overLap = nonzero(logical_and(dataMat[:, item].A>0, dataMat[:, j].A>0))[0]
        # 若两者没有任何重合元素，则相似度为0且中止本次循环
        if len(overLap) == 0 : similarity = 0
        # 如果存在重合的物品，则基于这些重合物品计算相似度
        else : similarity = simMeas(dataMat[overLap, item], dataMat[overLap, j])
        # print 'the %d and %d similarity is : %f' % (item, j, similarity)
        # 随后相似度不断累加
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0 : return 0
    # 通过除以所有的评分总和，对上述相似度评分的乘积进行归一化。这使得评分值在0-5之间，
    # 而这些评分值则用于对预测值进行排序
    else : return ratSimTotal/simTotal

# 推荐引擎，会调用standEst()函数，产生最高的N个推荐结果。
# simMeas：相似度计算方法
# estMethod：估计方法
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst) :
    # 寻找未评级的物品，对给定用户建立一个未评分的物品列表
    unratedItems = nonzero(dataMat[user, :].A==0)[1]
    # 如果不存在未评分物品，退出函数，否则在所有未评分物品上进行循环
    if len(unratedItems) == 0 : return 'you rated everything'
    itemScores = []
    for item in unratedItems :
        # 对于每个未评分物品，通过调用standEst()来产生该物品的预测评分。
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        # 该物品的编号和估计得分值会放在一个元素列表itemScores
        itemScores.append((item, estimatedScore))
    # 寻找前N个未评级物品
    return  sorted(itemScores, key=lambda jj : jj[1], reverse=True)[:N] 

In [8]:
recommend(myMat, 1, estMethod=svdEst)

the 0 and 3 similarity is : 0.490950
the 0 and 5 similarity is : 0.484274
the 0 and 10 similarity is : 0.512755
the 1 and 3 similarity is : 0.491294
the 1 and 5 similarity is : 0.481516
the 1 and 10 similarity is : 0.509709
the 2 and 3 similarity is : 0.491573
the 2 and 5 similarity is : 0.482346
the 2 and 10 similarity is : 0.510584
the 4 and 3 similarity is : 0.450495
the 4 and 5 similarity is : 0.506795
the 4 and 10 similarity is : 0.512896
the 6 and 3 similarity is : 0.743699
the 6 and 5 similarity is : 0.468366
the 6 and 10 similarity is : 0.439465
the 7 and 3 similarity is : 0.482175
the 7 and 5 similarity is : 0.494716
the 7 and 10 similarity is : 0.524970
the 8 and 3 similarity is : 0.491307
the 8 and 5 similarity is : 0.491228
the 8 and 10 similarity is : 0.520290
the 9 and 3 similarity is : 0.522379
the 9 and 5 similarity is : 0.496130
the 9 and 10 similarity is : 0.493617


[(4, 3.344714938469228), (7, 3.329402072452697), (9, 3.328100876390069)]

In [9]:
recommend(myMat, 1, estMethod=standEst)

[(6, 3.3333333333333335), (9, 3.3333333333333335), (0, 3.0)]

In [ ]:
# SVD用在图像压缩也很不错
